In [ ]:
# Uninstall the manually specified torch versions
!pip uninstall -y torch torchvision torchaudio

# Install transformers and its dependencies
!pip install transformers requests beautifulsoup4 pandas numpy

# Install torch with the correct index URL for CUDA 11.8.
# This lets pip find a compatible torch version for the installed transformers.
!pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118




Found existing installation: torch 2.1.0+cu118
Uninstalling torch-2.1.0+cu118:
  Successfully uninstalled torch-2.1.0+cu118
Found existing installation: torchvision 0.16.0+cu118
Uninstalling torchvision-0.16.0+cu118:
  Successfully uninstalled torchvision-0.16.0+cu118
Found existing installation: torchaudio 2.1.0+cu118
Uninstalling torchaudio-2.1.0+cu118:
  Successfully uninstalled torchaudio-2.1.0+cu118
Looking in indexes: https://download.pytorch.org/whl/cu118
  Using cached https://download.pytorch.org/whl/cu118/torch-2.7.0%2Bcu118-cp311-cp311-manylinux_2_28_x86_64.whl.metadata (28 kB)
  Using cached https://download.pytorch.org/whl/sympy-1.13.3-py3-none-any.whl.metadata (12 kB)
  Using cached https://download.pytorch.org/whl/triton-3.3.0-cp311-cp311-manylinux_2_27_x86_64.manylinux_2_28_x86_64.whl.metadata (1.5 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 955.6/955.6 MB 1.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 156.5/156.5 MB 7.0 MB/s eta 0:00:00
   ━━━━━━━

RuntimeError: Failed to import transformers.models.auto.tokenization_auto because of the following error (look up to see its traceback):
Failed to import transformers.generation.utils because of the following error (look up to see its traceback):
cannot import name 'justknobs_check' from 'torch._utils_internal' (/usr/local/lib/python3.11/dist-packages/torch/_utils_internal.py)

In [ ]:
# Re-import the necessary libraries
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch
import requests
from bs4 import BeautifulSoup
import re


In [ ]:

tokenizer = AutoTokenizer.from_pretrained('nlptown/bert-base-multilingual-uncased-sentiment')
model = AutoModelForSequenceClassification.from_pretrained('nlptown/bert-base-multilingual-uncased-sentiment')

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


model.safetensors:   0%|          | 0.00/669M [00:00<?, ?B/s]

In [ ]:

tokens = tokenizer.encode('Not that bad', return_tensors='pt')

In [ ]:
tokens

tensor([[  101, 10497, 10203, 12428,   102]])

In [ ]:
#tokenizer.decode(tokens[0])

In [ ]:
result = model(tokens)

In [ ]:
torch.argmax(result.logits)

tensor(2)

In [ ]:
int(torch.argmax(result.logits))+1

3

In [ ]:
r = requests.get('https://www.yelp.com/biz/social-brew-cafe-pyrmont')
soup = BeautifulSoup(r.text, 'html.parser')
regex = re.compile('.*comment.*')
results = soup.find_all('p', {'class':regex})
reviews = [result.text for result in results]

In [ ]:
reviews[0]

"Amazing ambience. Coffee was the best we've had in Sydney. The service and food also great. As must stop for breakfast."

In [ ]:
import numpy as np
import pandas as pd


In [ ]:
df = pd.DataFrame(np.array(reviews), columns=['review'])

In [ ]:
df.head()

,review
0,Amazing ambience. Coffee was the best we've ha...
1,Very cute coffee shop and restaurant. They hav...
2,Six of us met here for breakfast before our wa...
3,Ricotta hot cakes! These were so yummy. I ate ...
4,We came for brunch and they ran out of seven s...


In [ ]:
df.tail()

,review
5,We came for brunch twice in our week-long visi...
6,Great food amazing coffee and tea. Short walk ...
7,Great place with delicious food and friendly s...
8,The food was delicious. The ricotta pancakes w...
9,"Great service, lovely location, and really ama..."


In [ ]:
def sentiment_score(review):
  tokens = tokenizer.encode(review,return_tensors='pt')
  result = model(tokens)
  return int(torch.argmax(result.logits))+1


In [ ]:
sentiment_score(df['review'].iloc[1])

5

In [ ]:
df['sentiment']= df['review'].apply(lambda x: sentiment_score(x[:512]))

In [ ]:
df

,review,sentiment
0,Amazing ambience. Coffee was the best we've ha...,5
1,Very cute coffee shop and restaurant. They hav...,4
2,Six of us met here for breakfast before our wa...,4
3,Ricotta hot cakes! These were so yummy. I ate ...,5
4,We came for brunch and they ran out of seven s...,2
5,We came for brunch twice in our week-long visi...,4
6,Great food amazing coffee and tea. Short walk ...,5
7,Great place with delicious food and friendly s...,5
8,The food was delicious. The ricotta pancakes w...,4
9,"Great service, lovely location, and really ama...",5


In [ ]:
df['review'].iloc[5]

"We came for brunch twice in our week-long visit to Sydney. Everything on the menu not only sounds delicious, but is really tasty. It really gave us a sour taste of how bad breaky is in America with what's so readily available in Sydney!  Both days we went were Saturdays and there was a bit of a wait to be seated, the cafe is extremely busy for both dine-in and take-away. Service is fairly quick and servers are all friendly. The location is in Surrey Hills a couple blocks away from the bustling touristy Darling Harbor.The green smoothie is very tasty and refreshing. We tried the smoked salmon salad, the soft shell crab tacos, ricotta hotcakes, and the breaky sandwich. All were delicious, well seasoned, and a solid amount of food for the price. A definite recommend for anyone's trip into Sydney!"